# Pittsburgh Crime Analysis

We are working with three datasets. 

1. Pittsburgh police blotter dataset obtained from https://data.wprdc.org/dataset/pittsburgh-police-incident-blotter
The police blotter dataset is a listing of arrests in Pittsburgh documented shortly after an arrest is made. an arrest occurs but it does not include investigative, intelligence, nor treatment (medical) data. The information is "semi-refined" ie. a police report was taken, but it hadn't made its way through the court system nor the Uniform Crime Report.

2. Census 2010 Data obtained from http://www.pittsburghpa.gov/dcp/snap/raw_data
The Census data has demographic details of Pittsburgh. It breaks down the population by neighbourhoods and it gives the racial make up of each neighbourhood. It also gives the number of housing units in a neighbourhood and breaks it down by units occupied and vacant.

3. QUANDL API - http://static.quandl.com/zillow/hood_codes.csv 
We are obtaining Median Home prices for neighbourhoods within Pittsburgh using the Quandl API. Each neighbourhood has a code which the Quandl API uses. All neighbourhood codes in the US have been stored into a file hood_codes.csv. We will map our neighbourhood names with the names in this file and then call the API using the code, to get us a median list of house prices for that neighbourhood.

We combine these three datasets together to a final dataframe after cleaning and processing them.

We would be analysing what attributes are significant for each types of crime and attempt trend analysis to find trends for the crimes occuring in Pittsburgh. We would find the statistically significant attributes by running hypothesis tests.


First, we import the libraries. We will be using urllib2 and BeautifulSoup to parse the HTML, and pandas for the csv processing.

In [22]:
import urllib2
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import scipy.stats as st
import quandl
from collections import Counter

## Categorizing Crimes
Each crime in our dataset belongs to a category, or type, of crime. This category is labeled by the SECTION column. To map this SECTION number to the description of the type of crime, we need to parse the Pittsburgh crime website for this information. We did this by retrieving the HTML using urllib2, and use BeautifulSoup and regex pattern matching to obtain the section, or chapter, names and descriptions

In [23]:
web_page = urllib2.urlopen("http://www.legis.state.pa.us/WU01/LI/LI/CT/htm/18/18.htm").read()
soup = BeautifulSoup(web_page, "lxml")

In [24]:
descriptions = {}
for chapter in soup.find_all(text=re.compile('Chapter .* \xa0')):
    chapter_split = chapter.split()
    num = int(chapter_split[1][:-1])
    desc = " ".join(chapter_split[2:])
    descriptions[num] = desc
    
print descriptions

{1: u'General Provisions', 3: u'Culpability', 5: u'General Principles of Justification', 7: u'Responsibility (Reserved)', 9: u'Inchoate Crimes', 11: u'Authorized Disposition of Offenders', 13: u'Authority of Court in Sentencing (Transferred)', 21: u'Offenses Against the Flag', 23: u'General Provisions', 25: u'Criminal Homicide', 26: u'Crimes Against Unborn Child', 27: u'Assault', 29: u'Kidnapping', 30: u'Human Trafficking', 31: u'Sexual Offenses', 32: u'Abortion', 33: u'Arson, Criminal Mischief and Other Property Destruction', 35: u'Burglary and Other Criminal Intrusion', 37: u'Robbery', 39: u'Theft and Related Offenses', 41: u'Forgery and Fraudulent Practices', 43: u'Offenses Against the Family', 45: u'General Provisions', 47: u'Bribery and Corrupt Influence', 49: u'Falsification and Intimidation', 51: u'Obstructing Governmental Operations', 53: u'Abuse of Office', 55: u'Riot, Disorderly Conduct and Related Offenses', 57: u'Wiretapping and Electronic Surveillance', 59: u'Public Indece

We will load the csv file into a dataframe so we can process it.

In [25]:
Crime = pd.read_csv('Pittsburgh-Crime-Dataset.csv')
print Crime.dtypes
print len(Crime)

_id               int64
REPORT_NAME      object
CCR               int64
SECTION          object
DESCRIPTION      object
ARREST_TIME      object
ADDRESS          object
NEIGHBORHOOD     object
ZONE            float64
AGE             float64
GENDER           object
dtype: object
117717


We then need to filter out all of the rows with bad section data, that have sections that don't correspond to crime categories properly.

In [26]:
# Get only rows that have sections with numeric sections corresponding to the type of crime.
cleaned = Crime[Crime.apply(lambda x: x['SECTION'].isdigit() and len(x['SECTION'])>=3, axis=1)]
print len(cleaned)

81574


We remove the last 2 digits of the section to retrieve the broader section number, and create columns for this broad section number and crime description, saving this to a new csv.

In [27]:
cleaned['CrimeNum'] = cleaned['SECTION'].apply(lambda x: int(x[:-2]))
cleaned = cleaned[cleaned.apply(lambda x: x['CrimeNum'] in descriptions, axis=1)]
cleaned['CrimeDescription'] = cleaned['CrimeNum'].apply(lambda x: descriptions[x])
print len(cleaned)

C:\Users\Rishub_2\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


72753


We can run interesting analysis now that we have the crimes categorized. For example, we can see the most common crimes that happened in Pittsburgh, the most common being Assault.

In [32]:
s=0
d={}
for (ind,sec) in cleaned.groupby('CrimeNum'):
    d[descriptions[ind]] = len(sec)
    s+=len(sec)
top5crimes = Counter(d).most_common(5)
print top5crimes

# Crime Victims are "crimes" involving a self-inflicted injury, mental illness, false alarm, etc.

[(u'Assault', 12660), (u'Theft and Related Offenses', 11248), (u'Robbery', 9778), (u'Arson, Criminal Mischief and Other Property Destruction', 7519), (u'Crime Victims', 5518)]


## Adding Demographical Data

We now use the Census data to extract demographical data to add to the crime data.

In [33]:
Crime = cleaned
Census= pd.read_csv('CensusData.csv')
Census=Census.rename(columns = {'HOOD':'NEIGHBORHOOD'})

The Census dataframe has multiple rows for each neighbourhood divided into blocks. We want aggregated statistics for each neighbourhood. The following code groups and aggregates rows together based on the neighbourhood name.

In [34]:
hood = Census.groupby(['NEIGHBORHOOD'])
censusAggdf = hood.agg(np.sum)

Renaming columns and deleting columns that are insignificant for our analysis either due to very large number of missing values, meaningless identification codes or values that are too small  compared to the corresponding values.

In [35]:
censusAggdf = pd.DataFrame(censusAggdf)

del censusAggdf['TRACT']
del censusAggdf['CENSUS CODE']
del censusAggdf['BLOCK GRP']
del censusAggdf['ONERACE']
del censusAggdf['ONERACE18+']
del censusAggdf['2+RACES+']
del censusAggdf['2+RACES18+']
del censusAggdf['OTHER']
del censusAggdf['OTHER18+']


censusAggdf=censusAggdf.rename(columns = {'OCCUP':'occupied','VACANT':'vacant'})

censusAggdf.reset_index(level=0, inplace=True)

We join the two dataframes based on the neighbourhood. 

In [36]:
pittdf = pd.merge(Crime,censusAggdf,  how='inner', on=['NEIGHBORHOOD'])

The resulting dataset looks like this.

In [37]:
print pittdf.head()

     _id  REPORT_NAME       CCR SECTION  \
0  53961  OFFENSE 2.0  15158500     903   
1  53962  OFFENSE 2.0  15158500    6106   
2  53963  OFFENSE 2.0  15158500    3925   
3  55262  OFFENSE 2.0  15162081    9490   
4  54418       ARREST  15147051    3304   

                                     DESCRIPTION          ARREST_TIME  \
0                           Criminal Conspiracy.  2015-08-22T23:59:00   
1  Firearms not to be Carried without a License.  2015-08-22T23:59:00   
2                     Receiving Stolen Property.  2015-08-22T23:59:00   
3                  Missing Persons (18 and Over)  2015-08-27T23:10:00   
4                             Criminal Mischief.  2015-08-24T01:48:00   

                  ADDRESS      NEIGHBORHOOD  ZONE   AGE   ...   WHITE18+  \
0  400 block East Commons  Allegheny Center   1.0   NaN   ...        361   
1  400 block East Commons  Allegheny Center   1.0   NaN   ...        361   
2  400 block East Commons  Allegheny Center   1.0   NaN   ...        361  

## Adding Median House Prices by Neighbourhood
We will use the Quandl API to get median house prices for each neighbourhood. Each neighbourhood has a code which the Quandl API uses. All neighbourhood codes in the US have been stored into a file hood_codes.csv. We will map our neighbourhood names with the names in this file and then call the API using the code, to get us a median list of house prices for that neighbourhood.

In [38]:
apiKey = "nzKQCTmvvKxw5qLxsYxp"
quandl.ApiConfig.api_key = apiKey
df_all = pittdf
all_neighbourhoods = np.unique(df_all["NEIGHBORHOOD"])
hood_codes = pd.read_csv("hood_codes.csv")
hood_codes = hood_codes[["Region","City|Code","State"]]
hood_codes = hood_codes[hood_codes["State"]=="Pittsburgh"]

The hood_codes are of the type City|Code. We split this and just keep the value of Code.

In [39]:
code_list = []
for i in hood_codes["City|Code"]:
    code_number = i.split("|")
    code_list.append(code_number[1])
hood_codes["City|Code"] = code_list

Now, we are only interested in the codes in our dataset. So we form an intersection of all the codes that are in our dataset (clean_hoods) and the ones in the Quandl api (hood_codes) and then zip that with the name of the neighbourhoods.

In [16]:
clean_hoods = []
for i in all_neighbourhoods:
    i = i.strip()
    clean_hoods.append(i)
    
hood_set1 = set(clean_hoods)
hood_set2 = set(hood_codes["Region"])
#missingHoods = hood_set1 - hood_set2
found_hoods = hood_set1.intersection(hood_set2)

all_list = []
for i in found_hoods:
    all_list.append(hood_codes[hood_codes["Region"] == i]["City|Code"])
    
code_list = []
for i in all_list:
    z = i.get_values()
    code_list.extend(z)

named_codes = zip(code_list,found_hoods)

You can view how to call the Quandl API here - https://blog.quandl.com/api-for-housing-data
We add the code number into the API get call of the form ZILL/N(code)_MSP and store the results of each call into a list. In case that there are no sale prices available for that HTTP, it will throw an error. We catch the error and save its median sale price as no_data.

In [19]:
msp_list = []
counter=0
for i in code_list:
    try:
        median_sale_price = "ZILL/N"+i+"_MSP"
        msp_df = quandl.get(median_sale_price)
        msp_list.append(msp_df)
    except:
        msp_list.append("no_data")
        counter+=1

53


Save the name of the hoods and the MSP list into a dictionary.

In [42]:
values = zip(found_hoods,msp_list)
median_dict = {}
for i,v in values:
    if(type(v)!=str):
        median_dict[i]=v
        

Take the latest value for the neighbourhood as our MSP and store it in the dict.

In [43]:
for k,v in median_dict.iteritems():
    v = v.tail(1)
    v = v.get_values()
    median_dict[k] = v[0][0]

In [47]:
finalDF = pittdf[pittdf.apply(lambda x: x['NEIGHBORHOOD'] in median_dict, axis=1)]
finalDF['median_val'] = finalDF['NEIGHBORHOOD'].apply(lambda x: median_dict[x])
print len(finalDF)

24538


C:\Users\Rishub_2\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


This is what the final dataframe looks like:

In [49]:
print finalDF.head()

        _id  REPORT_NAME       CCR SECTION  \
5084  53990  OFFENSE 2.0  15158254    3929   
5085  54218  OFFENSE 2.0  15158953    3745   
5086  54727  OFFENSE 2.0  15159964    3743   
5087  54736  OFFENSE 2.0  15160133    3745   
5088  54986  OFFENSE 2.0  15161088    3745   

                                            DESCRIPTION          ARREST_TIME  \
5084                                      Retail Theft.  2015-08-22T17:30:00   
5085  Accidents Involving Damage to Unattended Veh.o...  2015-08-23T10:00:00   
5086  Accidents Involving Damage to Attended Veh. or...  2015-08-25T00:02:00   
5087  Accidents Involving Damage to Unattended Veh.o...  2015-08-25T09:00:00   
5088  Accidents Involving Damage to Unattended Veh.o...  2015-08-26T03:45:00   

                      ADDRESS           NEIGHBORHOOD  ZONE  AGE     ...      \
5084     4100 block Butler St  Central Lawrenceville   2.0  NaN     ...       
5085        100 block 43rd St  Central Lawrenceville   2.0  NaN     ...       
5086 